In [95]:
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

In [96]:
from underthesea import word_tokenize

In [97]:
stopwords_path = "Data/Data_ref/vietnamese-stopwords.txt"

In [98]:
# df = pd.read_csv('Data/Data_dat_dai/data_after_preprocessing.csv', index_col=0)
# df.head()

In [99]:
# X_train, X_test, Y_train, Y_test= model_selection.train_test_split(df['text'],df['label'],test_size=0.2,random_state=1)

In [100]:
def read_splitted_data():

    df_train = pd.read_csv('Data/Data_dat_dai/df_train_after.csv')
    df_test = pd.read_csv('Data/Data_dat_dai/df_test_after.csv')


    X_train = df_train[['text']]
    Y_train = df_train[['label']]

    X_test = df_test[['text']]
    Y_test = df_test[['label']]   

    return X_train, Y_train, X_test, Y_test

In [101]:
def tf_idf_config(X_train, X_test):

    Tfidf_vect = TfidfVectorizer(max_features=1000)
    Tfidf_vect.fit(X_train['text'].to_list())
    tfidf_train = Tfidf_vect.transform(X_train['text'])
    tfidf_test = Tfidf_vect.transform(X_test['text'])

    tfidf_train = tfidf_train.toarray()
    tfidf_test = tfidf_test.toarray()

    return Tfidf_vect, tfidf_train, tfidf_test

In [102]:
def model_selection(model_name, tfidf_train, tfidf_test, Y_train, Y_test):
    if (model_name == "SVM"):
        model = svm.SVC(C=1.0, kernel='linear', gamma='auto', degree = 3)
        model.fit(tfidf_train,Y_train)
        # predict the labels on validation dataset
        predictions_SVM = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)  
        print(classification_report(Y_test,predictions_SVM)) 
        return model    

    elif (model_name == "LGBM"):

        model = LGBMClassifier()
        model.fit(tfidf_train,Y_train)
        predictions_lgbm = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, Y_test)*100)
        print(classification_report(Y_test,predictions_lgbm))

        return model

    elif (model_name == "Random Forest"):

        model = RandomForestClassifier(n_estimators=1000,min_samples_leaf=2,
                                        min_samples_split= 10, max_features = 'sqrt', criterion = 'entropy', bootstrap= True,
                                      random_state=25)
        model.fit(tfidf_train,Y_train) 
        predictions_rf = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("rf Accuracy Score -> ",accuracy_score(predictions_rf, Y_test)*100)
        print(classification_report(Y_test,predictions_rf))

        return model
    else:
        print("No invalid model selected, try again")
        return 0


In [103]:
X_train, Y_train, X_test, Y_test = read_splitted_data()
Tfidf_vect, tfidf_train, tfidf_test = tf_idf_config(X_train, X_test)
selected = model_selection(model_name = "Random Forest", tfidf_train = tfidf_train, tfidf_test = tfidf_test, Y_train = Y_train, Y_test = Y_test)

C:\Users\admin\AppData\Local\Temp\ipykernel_4276\3729757558.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(tfidf_train,Y_train)


rf Accuracy Score ->  87.71929824561403
              precision    recall  f1-score   support

          -1       0.90      0.78      0.84        46
           1       0.86      0.94      0.90        68

    accuracy                           0.88       114
   macro avg       0.88      0.86      0.87       114
weighted avg       0.88      0.88      0.88       114



### Test Prediction

In [104]:
def lower_df(df):
    df['text'] = df['text'].str.lower()
    return df

In [105]:
def segmentation(df):


    '''UNDER THE SEA'''
    
    list_text = df['text'].to_list()
    #print(list_text[0])
    for i in range(len(list_text)):
        list_text[i] = word_tokenize(list_text[i], format='text')
    
    df['text'] =  list_text
    return df

In [106]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [107]:
def remove_stopwords(df):
    stopwords = get_stopwords_list(stopwords_path)

    list_text = df['text'].tolist()

    results = []
    for text in list_text:
        tmp = text.split(' ')
        for stop_word in stopwords:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))

    df['text'] = results
    return df

In [108]:
def data_preprocessing(df):
    df = lower_df(df)
    df = segmentation(df)
    df = remove_stopwords(df)

    return df

In [109]:
def predict_sample(tmp_text):
    input = tmp_text
    list_input = [input]
    test_df = pd.DataFrame()
    test_df['text'] = list_input
    test_df = data_preprocessing(test_df)

    test_df = Tfidf_vect.transform(test_df['text'])
    test_df = test_df.toarray()
    prediction = selected.predict(test_df)   

    return prediction



In [110]:
input  = """Gia đình người nước ngoài tìm hiểu dự án để mua nhà ở tại Khu đô thị Phú Mỹ Hưng (TP.HCM). Ảnh: Lê Toàn Dự thảo đang bỏ ngỏ Tuần qua, Dự thảo Luật Đất đai (sửa đổi) đã được đặt lên bàn nghị sự của Chính phủ trong phiên họp chuyên đề về xây dựng pháp luật do Thủ tướng Chính phủ Phạm Minh Chính chủ trì. Một trong những nội dung được các thành viên Chính phủ quan tâm thảo luận là quyền của người nước ngoài liên quan đến đất đai tại Việt Nam. Đây cũng chính là vấn đề được nhiều doanh nhân, chuyên gia quan tâm, góp ý, bởi đang có vướng mắc với một số luật khác. Cụ thể, Điều 159, Luật Nhà ở quy định tổ chức, cá nhân người nước ngoài là một trong những đối tượng được sở hữu nhà ở tại Việt Nam. Khoản 2, Điều 14, Luật Kinh doanh bất động sản quy định người Việt Nam định cư ở nước ngoài, tổ chức, cá nhân nước ngoài được thuê các loại bất động sản để sử dụng, được mua, thuê nhà ở theo quy định của pháp luật về nhà ở. “Nội dung về quyền sở hữu nhà ở tại Việt Nam của tổ chức, cá nhân nước ngoài đã được Bộ Xây dựng báo cáo và Bộ Chính trị thống nhất đưa vào Luật Nhà ở”, Bộ Tài nguyên và Môi trường nêu rõ trong hồ sơ Dự án Luật Đất đai (sửa đổi). Tuy nhiên, Điều 5, Luật Đất đai hiện hành quy định về người sử dụng đất chỉ quy định tổ chức nước ngoài được thực hiện dự án nhà ở tại Việt Nam, không quy định cá nhân là người nước ngoài được sử dụng đất ở tại Việt Nam. Để đảm bảo tính thống nhất, đồng bộ của hệ thống pháp luật, cần sửa đổi Luật Đất đai theo hướng bổ sung đối tượng cá nhân nước ngoài được sở hữu nhà ở tại Việt Nam theo pháp luật về nhà ở thì được sử dụng đất theo pháp luật về đất đai. Nhưng, theo Bộ Tài nguyên và Môi trường (cơ quan chủ trì soạn thảo Dự án Luật Đất đai sửa đổi), đây không phải là phương án duy nhất. “Hầu hết các nước trên thế giới đều có các quy định nhằm hạn chế hoặc kiểm soát các quyền và nghĩa vụ của người nước ngoài liên quan đến đất đai, bất động sản ở các mức độ khác nhau. Trong bối cảnh đất đai ở nước ta thuộc sở hữu toàn dân, nên việc trao quyền sử dụng đất cho người nước ngoài cần thận trọng và phải trên cơ sở quan điểm chỉ đạo của Trung ương. “Trong quá trình tổng kết Nghị quyết số 19-NQ/TW, vấn đề này đã được đưa vào báo cáo, tuy nhiên, sau đó đã có chỉ đạo đưa ra”, cơ quan chủ trì soạn thảo nêu lý do cần thận trọng. Từ giải thích này, Bộ Tài nguyên và Môi trường đề xuất hai phương án xử lý. Phương án 1: Bộ Xây dựng và Bộ Tài nguyên và Môi trường phối hợp trong quá trình dự thảo luật tiếp tục báo cáo Ban Cán sự đảng Chính phủ, Đảng đoàn Quốc hội xem xét báo cáo Bộ Chính trị cho phép người nước ngoài sở hữu nhà ở tại Việt Nam được sử dụng đất tại Việt Nam. Theo đó, Dự thảo sẽ sửa đổi quy định về người sử dụng đất tại Điều 5, Luật Đất đai và các quy định tại chương quy định về quyền và nghĩa vụ của người sử dụng đất để đảm bảo tương thích sau khi có ý kiến chỉ đạo của cấp có thẩm quyền. Phương án 2 được tính đến là, Nghị quyết số 18-NQ/TW đã có quy định về việc sử dụng không gian ngầm, không gian trên không. Vì vậy, có thể xem xét giải quyết vấn đề sử dụng đất của tổ chức, cá nhân nước ngoài thông qua bổ sung quy định mới liên quan đến người nước ngoài được sử dụng không gian xây dựng công trình gắn liền với đất giống như kinh nghiệm của một số nước hiện nay cho phép đất của một người nhưng tài sản trên đất thuộc sở hữu của người khác. Tuy nhiên, Dự thảo cũng chưa có quy định về vấn đề này, tức là vẫn đang bỏ ngỏ cả hai phương án. Theo quy định tại Điều 6, Dự thảo, “cá nhân người nước ngoài” không được xem là người sử dụng đất, không được Nhà nước giao đất, cho thuê đất, công nhận quyền sử dụng đất, nhận chuyển quyền sử dụng đất; thuê lại đất trong khu công nghiệp, cụm công nghiệp, khu công nghệ cao. Cần thống nhất Thông tin từ cuộc họp chuyên đề xây dựng pháp luật của Chính phủ tháng 8/2022 cho biết, đối với nội dung về tiếp cận đất đai của tổ chức kinh tế có vốn đầu tư nước ngoài, Thủ tướng yêu cầu bổ sung quy định về hạn chế và kiểm soát được việc tiếp cận các khu vực trọng yếu, nhạy cảm về quốc phòng, an ninh. Ngoài ra, Thủ tướng cũng chỉ đạo cân nhắc bổ sung quy định người nước ngoài thuộc đối tượng được phép sở hữu nhà ở theo pháp luật về nhà ở thì được quyền sử dụng đất ở để có căn cứ cấp giấy chứng nhận quyền sử dụng đất và tài sản gắn liền với đất nhằm thống nhất với Dự án Luật Nhà ở (sửa đổi) về đối tượng được sở hữu nhà ở là người nước ngoài. Xem xét lại quy định về người sử dụng đất đối với người nước ngoài để đảm bảo tính đồng bộ trong hệ thống pháp luật cũng là quan điểm của Liên đoàn Thương mại và Công nghiệp Việt Nam (VCCI) khi tham gia thẩm định Dự thảo Luật Đất đai (sửa đổi). Giữ như quy định hiện hành, nếu người nước ngoài bán nhà cho người Việt Nam thì vô hình trung, quyền của người mua là người Việt Nam sẽ không được đảm bảo. Đó là điều bất hợp lý mà theo VCCI, cần sửa đổi. Vướng mắc ở quy định về quyền sử dụng đất của người nước ngoài tại Việt Nam cũng được ông Nguyễn Văn Đỉnh, chuyên gia pháp lý đất đai, đầu tư xây dựng, kinh doanh bất động sản đề cập khá sâu trong hội thảo đầu tiên góp ý về Dự thảo. Ông Đỉnh phân tích, về mặt logic, người bán (người nước ngoài) không có quyền sử dụng đất thì đương nhiên người mua cũng không có quyền sử dụng đất (bởi không được nhận chuyển giao quyền này từ người bán). Như thế, người Việt Nam mua nhà ở của người nước ngoài chịu quy chế pháp lý như người nước ngoài (chỉ có quyền sở hữu nhà ở, mà không gắn với quyền sử dụng đất). Mặt khác, trong tờ trình đề nghị xây dựng Luật Nhà ở sửa đổi (dự kiến thông qua cùng thời điểm với Luật Đất đai sửa đổi), Bộ Xây dựng đề xuất tập trung giải quyết 8 nhóm chính sách lớn, đầu tiên là chính sách về sở hữu nhà ở: “Tiếp tục chính sách khuyến khích, tạo điều kiện thuận lợi cho cá nhân, tổ chức nước ngoài được mua và sở hữu nhà ở tại Việt Nam, phù hợp với thông lệ quốc tế, thu hút đầu tư, đồng thời vẫn bảo đảm an ninh, quốc phòng”. Nhưng chính sách khuyến khích cá nhân nước ngoài mua nhà ở sẽ không thể thực hiện trọn vẹn nếu không giải quyết được tận gốc vấn đề chứng nhận quyền sở hữu. “Để giải quyết tận gốc, Luật Đất đai (sửa đổi) cần thiết phải ghi nhận quyền sử dụng đất của cá nhân nước ngoài”, vị chuyên gia này góp ý. Sau khi được Ủy ban Kinh tế thẩm tra, Ủy ban Thường vụ Quốc hội cho ý kiến, xin ý kiến đại biểu chuyên trách, Dự thảo Luật Đất đai (sửa đổi) sẽ được trình Quốc hội tại Kỳ họp thứ tư (tháng 10/2022). Tránh lạm dụng trong thu hồi đất Sửa đổi Luật Đất đai, một trong các vấn đề được đặc biệt quan tâm là cơ chế thu hồi đất. Về vấn đề này, Thủ tướng yêu cầu, đối với các trường hợp Nhà nước thu hồi đất, cần cụ thể hóa trong Dự thảo các tiêu chí, điều kiện đối với trường hợp Nhà nước thu hồi đất, tránh tình trạng lạm dụng các trường hợp phát triển kinh tế để thu hồi đất, gây bức xúc trong nhân dân, nhất là thu hồi cho dự án nhà ở thương mại. Thủ tướng cũng lưu ý, đối với trường hợp dự án khu đô thị, dự án nhà ở thương mại vừa thuộc đối tượng thu hồi đất, vừa thuộc đối tượng nhận chuyển nhượng, thì cần có quy định cụ thể khi nào thực hiện thu hồi đất, khi nào thực hiện nhận chuyển nhượng.
"""


output = predict_sample(input)

if (output[0] == -1):
    print("Bai viet co kha nang tieu cuc cao !!!")   
else:
    print("Bai viet co kha nang tieu cuc thap :3")

Bai viet co kha nang tieu cuc thap :3


In [111]:
input  = """Cơ quan Cảnh sát Điều tra Công an tỉnh Khánh Hòa vừa đề nghị Viện Kiểm sát Nhân dân tỉnh Khánh Hòa truy tố các ông Nguyễn Chiến Thắng, cựu Chủ tịch UBND tỉnh Khánh Hòa; Đào Công Thiên, cựu Phó Chủ tịch UBND tỉnh và Võ Tấn Thái, cựu Giám đốc Sở Tài nguyên và Môi trường về tội “Vi phạm các quy định về quản lý đất đai” xảy ra tại dự án Nha Trang Golden Gate tại địa chỉ 28E Trần Phú, phường Vĩnh Nguyên, thành phố Nha Trang. Dự án Tổ hợp khách sạn và căn hộ du lịch cao cấp Nha Trang Golden Gate rộng hơn 20 ngàn m2, có vị trí đắc địa, nằm trải dài trên đường Trần Phú, con đường ven biển đẹp nhất thành phố Nha Trang. Đây là khu đất trước đây do Công ty Điện lực Khánh Hòa sử dụng. Tháng 2/2013, UBND tỉnh Khánh Hòa yêu cầu Điện lực Khánh Hòa di dời các cơ sở ra khỏi khu đất. Đến nay, Khu đất 28E, đường Trần Phú, TP Nha Trang vẫn bỏ hoang Sau đó, Ban Thường vụ Tỉnh ủy Khánh Hòa đã có văn bản thống nhất giao khu vực này cho dự án Nha Trang Golden Gate do Công ty TNHH Đỉnh Vàng Nha Trang làm chủ đầu tư có diện tích trên 14.200 m2. Mục đích dự án là xây dựng khu tổ hợp khách sạn và căn hộ du lịch cao cấp có tiêu chuẩn 5 sao với tổng mức đầu tư 1.250 tỷ đồng. Đến tháng 2/2016, UBND tỉnh Khánh Hòa có quyết định giao đất, cho thuê đất đối với chủ đầu tư, tổng diện tích dự án tăng lên hơn 20.000 m2. Đặc biệt, dự án này có diện tích đất không thu tiền sử dụng đất lên đến hơn 7.000 m2, gồm đất công viên, giao thông, sân bãi. Sau khi xây dựng, chủ đầu tư bàn giao lại cho nhà nước quản lý. Tháng 9/2016, Cục Thuế tỉnh Khánh Hòa có quyết định miễn tiền thuế đất phải nộp lên đến 3,4 tỷ đồng. Chủ đầu tư chỉ phải nộp tiền thuê đất để thực hiện dự án Nha Trang Golden Gate khoảng 75,9 tỷ đồng. Khu đất vàng 28E Trần Phú nằm bên bờ biển vịnh Nha Trang Sau khi chủ đầu tư hoàn tất nghĩa vụ tài chính, tháng 4/2018, UBND tỉnh Khánh Hòa đã cấp giấy chứng nhận Quyền sử dụng đất, Quyền sở hữu nhà ở và tài sản gắn liền với đất trên diện tích hơn 12.900 m2; trong đó có hơn 4.600 m2 đất ở tại đô thị, hơn 8 ngàn m2 đất thương mại dịch vụ. Theo Kết luận điều tra, việc ông Nguyễn Chiến Thắng ký cấp Giấy chứng nhận đầu tư, điều chỉnh và thỏa thuận phương án kiến trúc cho dự án là hành vi lạm quyền, biến một dự án chưa được Thủ tướng chấp thuận chủ trương đầu tư trở thành một dự án được công nhận, được thỏa thuận phương án kiến trúc. Qua các quyết định này, doanh nghiệp được giao đất, cho thuê đất không đúng căn cứ, không đúng đối tượng, không đúng hình thức của Luật Đất đai 2013. Trong khi đó, với tư cách Phó Chủ tịch UBND tỉnh Khánh Hòa, ông Đào Công Thiên đã có hành vi giao đất, cho thuê đất là trái với pháp luật đất đai. Ngoài ra, việc giao đất, cho thuê đất không qua đấu giá quyền sử dụng đất hoặc đấu thầu dự án có sử dụng đất là trái pháp luật về nhà ở. Ông Nguyễn Chiến Thắng, cựu Chủ tịch UBND tỉnh Khánh Hòa đã bị khởi tố trong một vụ án khác Với vai trò là Giám đốc Sở Kế hoạch và Đầu tư, sau đó là Giám đốc Sở Tài nguyên và Môi trường, ông Võ Tấn Thái đã ký các Tờ trình tham mưu UBND tỉnh chỉ định nhà đầu tư dự án và ký hàng loạt tờ trình khác trái với Luật Đất đai 2013.
"""

output = predict_sample(input)

if (output[0] == -1):
    print("Bai viet co kha nang tieu cuc cao !!!")   
else:
    print("Bai viet co kha nang tieu cuc thap :3")

Bai viet co kha nang tieu cuc cao !!!


In [112]:
import pickle

In [113]:
pickle.dump(selected, open('Data/model/rf.pkl', 'wb'))

In [116]:
pickle.dump(Tfidf_vect, open("Data/model/tfidf.pickle", "wb"))

In [117]:
def text_to_df(tmp_text):
    input = tmp_text
    list_input = [input]
    test_df = pd.DataFrame()
    test_df['text'] = list_input    
    return test_df

def predict_sample(test_df, selected, Tfidf_vect):

    test_df = data_preprocessing(test_df)
    test_df = Tfidf_vect.transform(test_df['text'])
    test_df = test_df.toarray()
    prediction = selected.predict(test_df)   

    return prediction

In [120]:
pickled_model = pickle.load(open('Data/model/lgbm.pkl', 'rb'))
Tfidf_vect = pickle.load(open('Data/model/tfidf.pickle', 'rb'))
# input  = """Bài 1: Quy hoạch chưa sát thực tế Công tác lập quy hoạch, kế hoạch sử dụng đất thời kỳ đầu (giai đoạn 2016-2020) của một số địa phương thời gian qua chất lượng chưa cao, chưa sát với tình hình thực tế và quá trình phát triển kinh tế - xã hội. Việc một số quy hoạch đã công bố, thời gian kéo dài, không triển khai hoặc chỉ triển khai một phần, không điều chỉnh quy hoạch, làm ảnh hưởng đến quyền lợi người dân trong vùng dự án. Bên cạnh đó, theo quy định, các dự án, công trình đưa vào kế hoạch sử dụng đất hàng năm của địa phương nhưng nếu quá 3 năm không triển khai thực hiện thì người sử dụng đất được quyền sử dụng đất của mình. Tuy nhiên, có một số công trình, dự án tiếp tục đưa vào kỳ sử dụng đất tiếp theo, gây thiệt thòi cho người sử dụng đất trong 3 năm khi không khai thác, thu được lợi nhuận trên mảnh đất của mình. Định hướng và khái quát Cụ thể của việc đánh giá trên, tại huyện Đầm Dơi cho thấy, trong quy hoạch sử dụng đất kỳ trước qua các năm 2016-2020, trên địa bàn huyện đã thực hiện được 263/672 hạng mục công trình, dự án (trong đó có 1 hạng mục thực hiện 1 phần của dự án đầu tư xây dựng hệ thống thuỷ lợi phục vụ nuôi thuỷ sản), đạt 39,14% về mặt tổng số công trình, dự án đã đăng ký, với tổng diện tích quy hoạch đã thực hiện là 591,18 ha/1.360,91 ha. Qua rà soát, trên địa bàn huyện Đầm Dơi có 6 công trình, dự án quá 3 năm nhưng chưa triển khai, trình cấp thẩm quyền xem xét huỷ bỏ theo quy định, gồm đường số 7, Khóm 4, thị trấn Đầm Dơi; lộ Tô Văn Mười nối dài (đoạn từ đường Nguyễn Tạo đến đường 30/4), Khóm 4, thị trấn Đầm Dơi; đê bao mương Chung Kiết - Khâu Mét; cụm dân cư Khóm 1 (phía Đông), thị trấn Đầm Dơi; khu dân cư (tái định cư) ấp Tân Long A, xã Tân Tiến; khu dân cư (tái định cư) ấp Thuận Long, xã Tân Tiến. Sau nhiều năm quy hoạch treo, lộ Tô Văn Mười nối dài (đoạn từ đường Nguyễn Tạo đến đường 30/4), Khóm 4, thị trấn Đầm Dơi, vẫn chưa có nguồn lực để thực hiện, được địa phương đưa ra khỏi quy hoạch trong giai đoạn tiếp theo. Ông Huỳnh Trung Quang, Chủ tịch UBND thị trấn Đầm Dơi, cho hay, trên địa bàn có 7 dự án tuyến lộ được quy hoạch từ năm 2010, nhưng đến nay chỉ làm được 1 tuyến. Xa hơn nữa, quy hoạch trung tâm thương mại của huyện nằm trên địa bàn thị trấn có từ năm 2005, nhưng đến nay vẫn chưa được thực hiện. “Nhiều năm không thực hiện, đồng nghĩa là quy hoạch treo, phải huỷ bỏ. Tuy nhiên, có những dự án không thể huỷ, như trung tâm thương mại, vì nhận thấy thật sự cần thiết, nên vẫn phải tiếp tục thực hiện”, ông Quang nêu thực tế. Ông Nguyễn Phương Bình, Phó chủ tịch UBND huyện Đầm Dơi, cho biết, công tác dự báo, xác định nhu cầu sử dụng đất trong quy hoạch kỳ trước của các ngành, đơn vị trên địa bàn huyện còn chậm, chưa rõ ràng và thống nhất về vị trí thực hiện, tên dự án, quy mô dự án. Do đó, trong quá trình tổ chức thực hiện gặp rất nhiều khó khăn do có điều chỉnh tên, quy mô, vị trí thực hiện dự án. Tuy công tác mời gọi đầu tư được tổ chức thực hiện tốt, nhưng vẫn còn nhiều cơ hội đầu tư bị bỏ lỡ. “Nguyên nhân chính là do vị trí quy hoạch dự án chưa đáp ứng được mong muốn của nhà đầu tư về vị trí tiếp giáp, tiềm năng phát triển..., ngược lại, nhà đầu tư đề xuất vị trí lại có đơn giá bồi thường giải phóng mặt bằng khá cao do thuộc các khu vực có nhiều hộ dân sinh sống, giá đất cao...”, ông Bình nhìn nhận thực tế dẫn đến việc triển khai các công trình, dự án được quy hoạch sử dụng đất thời gian qua trên địa bàn đạt kết quả thấp. Một vấn đề được lãnh đạo UBND huyện Đầm Dơi nêu thực tế trên địa bàn là công tác quy hoạch phát triển cụm công nghiệp tiến triển chậm, chưa khai thác hết tiềm năng của địa phương. Cụ thể ở đây là quy hoạch cụm công nghiệp Tân Thuận 35 ha, cụm công nghiệp Nguyễn Huân 50 ha, cụm công nghiệp thị trấn Đầm Dơi 35 ha… “Vị trí quy hoạch cụm công nghiệp trong quy hoạch kỳ trước nằm trong khu vực đô thị, dẫn đến giá trị bồi thường, hỗ trợ tái định cư cao; quy mô còn nhỏ, tiềm năng mở rộng không lớn do tiếp giáp với khu đô thị trung tâm, hạ tầng giao thông đấu nối chưa đảm bảo khả năng phát triển công nghiệp. Cùng với đó, công tác khảo sát, nghiên cứu lập quy hoạch chi tiết 1/500, đồ án quy hoạch các tỷ lệ còn chậm, chưa thực hiện đồng bộ với thời điểm xây dựng quy hoạch sử dụng đất. Nhiều dự án được quy hoạch chỉ mang tính định hướng, khái quát nhằm mục tiêu mời gọi đầu tư”, ông Bình trần tình. Được quy hoạch từ rất lâu (năm 2005), đến nay Trung tâm Thương mại huyện Đầm Dơi vẫn chưa được triển khai thực hiện. Cùng chung thực trạng Cũng với thực trạng trên nhưng có thời gian dài hơn, suốt giai đoạn 2011-2020, huyện Năm Căn chỉ thực hiện được 234/712 công trình, dự án đã đăng ký thực hiện quy hoạch (đạt 32,87%), với diện tích 13.282,98/20.546,74 ha. Qua rà soát, địa phương loại bỏ 324 công trình, dự án với diện tích trên 6.346 ha cho giai đoạn đến năm 2030, vì xét thấy không còn phù hợp với điều kiện phát triển kinh tế - xã hội của huyện. Tại huyện Trần Văn Thời, giai đoạn 2016-2020 đã triển khai thực hiện được 243/796 hạng mục công trình, dự án thuộc trường hợp thực hiện thủ tục thu hồi đất, giao đất, cho thuê và chuyển mục đích sử dụng đất, đạt 30,53% về tổng số công trình, dự án đã đăng ký, với tổng diện tích đã thực hiện là 348,65 ha/ 2.946,06 ha, đạt 13,83% về diện tích thực hiện. Bên cạnh các dự án đã được phê duyệt, trong kỳ quy hoạch trên, huyện Trần Văn Thời đã thực hiện hoàn thành 17 dự án được phê duyệt bổ sung với tổng diện tích 1.390,22 ha. Cũng trong thời kỳ quy hoạch này, địa phương đã điều chỉnh tạm ngưng thực hiện 184/796 hạng mục công trình, dự án, tổng diện tích 673,88 ha. Với những con số trên, ông Hồ Song Toàn, Phó chủ tịch UBND huyện Trần Văn Thời, cho rằng, kết quả thực hiện các công trình, dự án trong quy hoạch sử dụng đất đến năm 2020 trên địa bàn đáp ứng tốt các mục tiêu phát triển kinh tế - xã hội theo từng giai đoạn trong 10 năm qua. Thừa nhận về kết quả thực hiện chưa đạt tỷ lệ cao, tuy nhiên, ông Toàn khẳng định thành quả đạt được là sự thay đổi lớn về kết cấu hạ tầng, chất lượng cuộc sống của người dân ngày càng được cải thiện; nhiều công trình, dự án đã thực hiện và được đưa vào khai thác đạt hiệu quả cao. Dự án đầu tư xây dựng khu dân cư bờ Nam Sông Đốc, với diện tích 287,07 ha, là 1 trong 4 dự án đã đưa vào kế hoạch sử dụng đất hàng năm, nhưng không triển khai thực hiện nên huyện đề nghị huỷ bỏ và được HĐND tỉnh ban hành Nghị quyết số 08/2021/NQ-HĐND, ngày 4/12/2021 chấp thuận. Tại thành phố trung tâm của tỉnh, trong quy hoạch sử dụng đất kỳ trước qua các năm 2016-2020, trên địa bàn TP Cà Mau đã đăng ký thực hiện 486 hạng mục công trình, dự án. Kết quả đã thực hiện được 124/486 hạng mục công trình, dự án (trong đó, có 5 dự án đã thực hiện một phần và chuyển tiếp sang kỳ quy hoạch mới để tiếp tục thực hiện), đạt 25,51% về mặt tổng số công trình, dự án đã đăng ký với tổng diện tích quy hoạch đã thực hiện là 314,98 ha/4.134,42 ha. Ngoài ra, giai đoạn 2018-2021, thành phố đã điều chỉnh tạm ngưng thực hiện 61 hạng mục công trình, dự án với tổng diện tích 796,34 ha. Chuyển tiếp sang kỳ quy hoạch 2021-2030 tiếp tục thực hiện với 306 hạng mục công trình, dự án. “TP Cà Mau trình cấp thẩm quyền xem xét huỷ bỏ công trình dự án quá 3 năm nhưng chưa triển khai, có thể kể đến Dự án nhà ở công nhân Phường 8 (đường Vành đai Tây Nam thuộc địa bàn Phường 8 và xã Lý Văn Lâm); Dự án khu nhà ở công nhân lao động Công ty Minh Phú; Dự án đầu tư xây dựng kết cấu hạ tầng đường Nguyễn Mai...”, ông Bùi Tứ Hải, Phó chủ tịch UBND TP Cà Mau, thông tin. Đối với cấp tỉnh, con số thống kê cho thấy, danh mục các công trình, dự án theo Nghị quyết 84/NQ-CP, ngày 7/10/2019 của Chính phủ có 498 công trình, dự án. Trong đó, tỉnh đã triển khai lập thủ tục đất đai 82 công trình, dự án; trình HĐND tỉnh ban hành nghị quyết huỷ bỏ 36 công trình, dự án, với diện tích 591,32 ha; điều chỉnh 17 công trình, dự án, với diện tích 430,57 ha./. Trần Nguyên BÀI 2: GIAO CHỈ TIÊU LỆCH SO VỚI THỰC TẾ
# """
# validate_sample(input, pickled_model, Tfidf_vect)
df = pd.read_csv("Data/Data_dat_dai/unlabelled.csv")
df = df.sample(1000)
pred = predict_sample(df, pickled_model, Tfidf_vect)
df['label'] = pred
df.to_csv('auto_labelling.csv', encoding= "utf-8-sig")

KeyboardInterrupt: 